In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import pairwise

In [4]:
# Create Global Variables
background1 = None
background2 = None
accumulated_weight1 = 0.5
accumulated_weight2 =0.5
roi_top = 200
roi_bottom = 450
roi_right = 370
roi_left = 620
arearatio = 0 
expression = []
text_expression = ''
operators = ['+','-','*','/']
sol = None
thumb = False
new = '0'
newOp = ''
oldOp = ''

In [5]:
# Create the Clear Function
def clear_all():
    expression = []
    text_expression = ''
    new = '0'
    newOp = ''
    oldOp = ''
    sol = None
    return (expression,text_expression,new,newOp,oldOp,sol)

In [6]:
# Create the background detection
def background_identification1(frame,accumulated_weight):
    global background1
    if background1 is None:
        background1 = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background1,accumulated_weight1)

In [7]:
# Do the background again
def background_identification2(frame,accumulated_weight):
    global background2
    if background2 is None:
        background2 = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background2,accumulated_weight2)

In [8]:
# Segment the hand
def segment(frame,background,threshold_min = 30):
    diff = cv2.absdiff(background.astype('uint8'),frame)
    ret,thresholded = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    thresholded = cv2.morphologyEx(thresholded,cv2.MORPH_CLOSE,kernel)
    contours,hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours,key=cv2.contourArea)
        return (thresholded,hand_segment)